In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import warnings
import time
import sklearn.preprocessing

pd.set_option("display.max_columns", None)
warnings.filterwarnings(action='ignore')

serviceKey='bwUUK6MEA6GnqDpo2ZS9ChN4VZ1nkUPXrryU80XxUcLqxB62pIGIPWXNdyzqgT20BHrw8zxWWk9rqiSQwTrbnQ=='

p7000 ={'serviceKey' :serviceKey,
             'routeId':'200000112','stationId':'228000703'}
class Bus():
    def __init__(self,params):
        self.params=params
        self.bus_time=[]
        
    def repet(self):
        url = 'http://apis.data.go.kr/6410000/busarrivalservice/getBusArrivalItem'
        response = requests.get(url, params=self.params)
        soup=BeautifulSoup(response.text,'xml')
        return soup

        
    def current_buses(self): 
        url = 'http://apis.data.go.kr/6410000/buslocationservice/getBusLocationList'
        mod_params ={'serviceKey' : self.params['serviceKey'],
                  'routeId':self.params['routeId']}

        response=requests.get(url,params=mod_params)
        soup=BeautifulSoup(response.text,'xml')
        locations=sorted([int(x.stationSeq.text) for x in soup.find_all('busLocationList')])
        return locations


def trim_data(df,FileName):
    df['시간'] = df['시간'].apply(pd.to_datetime)
    df['hour']=df['시간'].dt.hour
    df['minute']=df['시간'].dt.minute
    df['second']=df['시간'].dt.second
    df['day'] = df['시간'].dt.weekday 

    mb = sklearn.preprocessing.MultiLabelBinarizer()
    if isinstance(df['운행중인 다른 버스들의 위치'].iloc[0], str):
        df['운행중인 다른 버스들의 위치'] = df['운행중인 다른 버스들의 위치'].apply(eval)
    entity = mb.fit_transform(df['운행중인 다른 버스들의 위치']) 
    lsts=pd.DataFrame(entity,columns=mb.classes_)
    lsts.columns=[f'station{x}' for x in lsts.columns]
    df=pd.concat([df,lsts],axis=1,sort=False).reset_index(drop=True)


    df=df.drop(["운행중인 다른 버스들의 위치"],axis=1)
    
    batch=df.index[df.arrived==1]
    df=df.iloc[:batch[-1]+1,:]
    #7/12 -----------------------------------
    batch=df.index[df.arrived==1]
    temp=0
    for x in batch:
        for y in range(temp,x): 
            df['arrived'][y]=(df['시간'][x]-df['시간'][y]).seconds
            temp=x+1
    for e in batch:
        df['arrived'][e]=0
    # 7/12 -----------------------------------

    df = df.set_index('시간')
    df.to_csv(f'./{FileName}')        # 동일 폴더에 파일이 저장될 거임
    return df


In [2]:
past=4
pBus=Bus(p7000)
for x in range(720): # 시간 설정
    soup=pBus.repet()
    try:
        result=int(soup.resultCode.text)
        arrived=0
        if past!=result:
            if result==0:
                arrived=1
        pBus.bus_time.append([soup.queryTime.text,pBus.current_buses(),arrived])
        past=result
        print(x,"번째 시도: ",pBus.bus_time)
        time.sleep(60)
    except Exception as e:
        continue

df=pd.DataFrame(pBus.bus_time,columns=['시간','운행중인 다른 버스들의 위치','arrived'])

mod_df=trim_data(df,'7000_Fri.csv') # 이름 설정
mod_df

0 번째 시도:  [['2022-08-12 11:57:33.391', [9, 17, 26, 32, 40, 50, 53, 61], 0]]
1 번째 시도:  [['2022-08-12 11:57:33.391', [9, 17, 26, 32, 40, 50, 53, 61], 0], ['2022-08-12 11:58:34.578', [10, 18, 26, 33, 40, 50, 54, 62], 0]]
2 번째 시도:  [['2022-08-12 11:57:33.391', [9, 17, 26, 32, 40, 50, 53, 61], 0], ['2022-08-12 11:58:34.578', [10, 18, 26, 33, 40, 50, 54, 62], 0], ['2022-08-12 11:59:34.749', [10, 18, 26, 33, 41, 51, 55, 63], 0]]
3 번째 시도:  [['2022-08-12 11:57:33.391', [9, 17, 26, 32, 40, 50, 53, 61], 0], ['2022-08-12 11:58:34.578', [10, 18, 26, 33, 40, 50, 54, 62], 0], ['2022-08-12 11:59:34.749', [10, 18, 26, 33, 41, 51, 55, 63], 0], ['2022-08-12 12:00:35.108', [11, 18, 27, 34, 41, 51, 55, 63], 0]]
4 번째 시도:  [['2022-08-12 11:57:33.391', [9, 17, 26, 32, 40, 50, 53, 61], 0], ['2022-08-12 11:58:34.578', [10, 18, 26, 33, 40, 50, 54, 62], 0], ['2022-08-12 11:59:34.749', [10, 18, 26, 33, 41, 51, 55, 63], 0], ['2022-08-12 12:00:35.108', [11, 18, 27, 34, 41, 51, 55, 63], 0], ['2022-08-12 12:01:40.686'

18 번째 시도:  [['2022-08-12 11:57:33.391', [9, 17, 26, 32, 40, 50, 53, 61], 0], ['2022-08-12 11:58:34.578', [10, 18, 26, 33, 40, 50, 54, 62], 0], ['2022-08-12 11:59:34.749', [10, 18, 26, 33, 41, 51, 55, 63], 0], ['2022-08-12 12:00:35.108', [11, 18, 27, 34, 41, 51, 55, 63], 0], ['2022-08-12 12:01:40.686', [1, 12, 19, 27, 35, 42, 51, 56, 63], 1], ['2022-08-12 12:02:44.357', [3, 12, 19, 27, 36, 42, 51, 57, 63], 0], ['2022-08-12 12:03:44.528', [3, 14, 20, 28, 37, 43, 52, 58, 65], 0], ['2022-08-12 12:04:51.825', [4, 14, 21, 28, 37, 44, 52, 60, 65], 0], ['2022-08-12 12:05:53.980', [5, 14, 21, 29, 37, 44, 52, 61, 65], 0], ['2022-08-12 12:06:54.167', [5, 16, 22, 29, 37, 45, 52, 61, 66], 0], ['2022-08-12 12:08:02.135', [6, 16, 23, 30, 38, 45, 53, 63, 66], 0], ['2022-08-12 12:09:03.322', [8, 16, 24, 31, 39, 45, 53, 63, 67], 0], ['2022-08-12 12:10:12.228', [8, 17, 24, 32, 40, 46, 54, 65, 68], 0], ['2022-08-12 12:11:12.524', [9, 18, 25, 32, 41, 47, 55, 65, 68], 0], ['2022-08-12 12:12:15.930', [9, 19,

25 번째 시도:  [['2022-08-12 11:57:33.391', [9, 17, 26, 32, 40, 50, 53, 61], 0], ['2022-08-12 11:58:34.578', [10, 18, 26, 33, 40, 50, 54, 62], 0], ['2022-08-12 11:59:34.749', [10, 18, 26, 33, 41, 51, 55, 63], 0], ['2022-08-12 12:00:35.108', [11, 18, 27, 34, 41, 51, 55, 63], 0], ['2022-08-12 12:01:40.686', [1, 12, 19, 27, 35, 42, 51, 56, 63], 1], ['2022-08-12 12:02:44.357', [3, 12, 19, 27, 36, 42, 51, 57, 63], 0], ['2022-08-12 12:03:44.528', [3, 14, 20, 28, 37, 43, 52, 58, 65], 0], ['2022-08-12 12:04:51.825', [4, 14, 21, 28, 37, 44, 52, 60, 65], 0], ['2022-08-12 12:05:53.980', [5, 14, 21, 29, 37, 44, 52, 61, 65], 0], ['2022-08-12 12:06:54.167', [5, 16, 22, 29, 37, 45, 52, 61, 66], 0], ['2022-08-12 12:08:02.135', [6, 16, 23, 30, 38, 45, 53, 63, 66], 0], ['2022-08-12 12:09:03.322', [8, 16, 24, 31, 39, 45, 53, 63, 67], 0], ['2022-08-12 12:10:12.228', [8, 17, 24, 32, 40, 46, 54, 65, 68], 0], ['2022-08-12 12:11:12.524', [9, 18, 25, 32, 41, 47, 55, 65, 68], 0], ['2022-08-12 12:12:15.930', [9, 19,

30 번째 시도:  [['2022-08-12 11:57:33.391', [9, 17, 26, 32, 40, 50, 53, 61], 0], ['2022-08-12 11:58:34.578', [10, 18, 26, 33, 40, 50, 54, 62], 0], ['2022-08-12 11:59:34.749', [10, 18, 26, 33, 41, 51, 55, 63], 0], ['2022-08-12 12:00:35.108', [11, 18, 27, 34, 41, 51, 55, 63], 0], ['2022-08-12 12:01:40.686', [1, 12, 19, 27, 35, 42, 51, 56, 63], 1], ['2022-08-12 12:02:44.357', [3, 12, 19, 27, 36, 42, 51, 57, 63], 0], ['2022-08-12 12:03:44.528', [3, 14, 20, 28, 37, 43, 52, 58, 65], 0], ['2022-08-12 12:04:51.825', [4, 14, 21, 28, 37, 44, 52, 60, 65], 0], ['2022-08-12 12:05:53.980', [5, 14, 21, 29, 37, 44, 52, 61, 65], 0], ['2022-08-12 12:06:54.167', [5, 16, 22, 29, 37, 45, 52, 61, 66], 0], ['2022-08-12 12:08:02.135', [6, 16, 23, 30, 38, 45, 53, 63, 66], 0], ['2022-08-12 12:09:03.322', [8, 16, 24, 31, 39, 45, 53, 63, 67], 0], ['2022-08-12 12:10:12.228', [8, 17, 24, 32, 40, 46, 54, 65, 68], 0], ['2022-08-12 12:11:12.524', [9, 18, 25, 32, 41, 47, 55, 65, 68], 0], ['2022-08-12 12:12:15.930', [9, 19,

34 번째 시도:  [['2022-08-12 11:57:33.391', [9, 17, 26, 32, 40, 50, 53, 61], 0], ['2022-08-12 11:58:34.578', [10, 18, 26, 33, 40, 50, 54, 62], 0], ['2022-08-12 11:59:34.749', [10, 18, 26, 33, 41, 51, 55, 63], 0], ['2022-08-12 12:00:35.108', [11, 18, 27, 34, 41, 51, 55, 63], 0], ['2022-08-12 12:01:40.686', [1, 12, 19, 27, 35, 42, 51, 56, 63], 1], ['2022-08-12 12:02:44.357', [3, 12, 19, 27, 36, 42, 51, 57, 63], 0], ['2022-08-12 12:03:44.528', [3, 14, 20, 28, 37, 43, 52, 58, 65], 0], ['2022-08-12 12:04:51.825', [4, 14, 21, 28, 37, 44, 52, 60, 65], 0], ['2022-08-12 12:05:53.980', [5, 14, 21, 29, 37, 44, 52, 61, 65], 0], ['2022-08-12 12:06:54.167', [5, 16, 22, 29, 37, 45, 52, 61, 66], 0], ['2022-08-12 12:08:02.135', [6, 16, 23, 30, 38, 45, 53, 63, 66], 0], ['2022-08-12 12:09:03.322', [8, 16, 24, 31, 39, 45, 53, 63, 67], 0], ['2022-08-12 12:10:12.228', [8, 17, 24, 32, 40, 46, 54, 65, 68], 0], ['2022-08-12 12:11:12.524', [9, 18, 25, 32, 41, 47, 55, 65, 68], 0], ['2022-08-12 12:12:15.930', [9, 19,

...

108 번째 시도:  [['2022-08-12 11:57:33.391', [9, 17, 26, 32, 40, 50, 53, 61], 0], ['2022-08-12 11:58:34.578', [10, 18, 26, 33, 40, 50, 54, 62], 0], ['2022-08-12 11:59:34.749', [10, 18, 26, 33, 41, 51, 55, 63], 0], ['2022-08-12 12:00:35.108', [11, 18, 27, 34, 41, 51, 55, 63], 0], ['2022-08-12 12:01:40.686', [1, 12, 19, 27, 35, 42, 51, 56, 63], 1], ['2022-08-12 12:02:44.357', [3, 12, 19, 27, 36, 42, 51, 57, 63], 0], ['2022-08-12 12:03:44.528', [3, 14, 20, 28, 37, 43, 52, 58, 65], 0], ['2022-08-12 12:04:51.825', [4, 14, 21, 28, 37, 44, 52, 60, 65], 0], ['2022-08-12 12:05:53.980', [5, 14, 21, 29, 37, 44, 52, 61, 65], 0], ['2022-08-12 12:06:54.167', [5, 16, 22, 29, 37, 45, 52, 61, 66], 0], ['2022-08-12 12:08:02.135', [6, 16, 23, 30, 38, 45, 53, 63, 66], 0], ['2022-08-12 12:09:03.322', [8, 16, 24, 31, 39, 45, 53, 63, 67], 0], ['2022-08-12 12:10:12.228', [8, 17, 24, 32, 40, 46, 54, 65, 68], 0], ['2022-08-12 12:11:12.524', [9, 18, 25, 32, 41, 47, 55, 65, 68], 0], ['2022-08-12 12:12:15.930', [9, 19

110 번째 시도:  [['2022-08-12 11:57:33.391', [9, 17, 26, 32, 40, 50, 53, 61], 0], ['2022-08-12 11:58:34.578', [10, 18, 26, 33, 40, 50, 54, 62], 0], ['2022-08-12 11:59:34.749', [10, 18, 26, 33, 41, 51, 55, 63], 0], ['2022-08-12 12:00:35.108', [11, 18, 27, 34, 41, 51, 55, 63], 0], ['2022-08-12 12:01:40.686', [1, 12, 19, 27, 35, 42, 51, 56, 63], 1], ['2022-08-12 12:02:44.357', [3, 12, 19, 27, 36, 42, 51, 57, 63], 0], ['2022-08-12 12:03:44.528', [3, 14, 20, 28, 37, 43, 52, 58, 65], 0], ['2022-08-12 12:04:51.825', [4, 14, 21, 28, 37, 44, 52, 60, 65], 0], ['2022-08-12 12:05:53.980', [5, 14, 21, 29, 37, 44, 52, 61, 65], 0], ['2022-08-12 12:06:54.167', [5, 16, 22, 29, 37, 45, 52, 61, 66], 0], ['2022-08-12 12:08:02.135', [6, 16, 23, 30, 38, 45, 53, 63, 66], 0], ['2022-08-12 12:09:03.322', [8, 16, 24, 31, 39, 45, 53, 63, 67], 0], ['2022-08-12 12:10:12.228', [8, 17, 24, 32, 40, 46, 54, 65, 68], 0], ['2022-08-12 12:11:12.524', [9, 18, 25, 32, 41, 47, 55, 65, 68], 0], ['2022-08-12 12:12:15.930', [9, 19

112 번째 시도:  [['2022-08-12 11:57:33.391', [9, 17, 26, 32, 40, 50, 53, 61], 0], ['2022-08-12 11:58:34.578', [10, 18, 26, 33, 40, 50, 54, 62], 0], ['2022-08-12 11:59:34.749', [10, 18, 26, 33, 41, 51, 55, 63], 0], ['2022-08-12 12:00:35.108', [11, 18, 27, 34, 41, 51, 55, 63], 0], ['2022-08-12 12:01:40.686', [1, 12, 19, 27, 35, 42, 51, 56, 63], 1], ['2022-08-12 12:02:44.357', [3, 12, 19, 27, 36, 42, 51, 57, 63], 0], ['2022-08-12 12:03:44.528', [3, 14, 20, 28, 37, 43, 52, 58, 65], 0], ['2022-08-12 12:04:51.825', [4, 14, 21, 28, 37, 44, 52, 60, 65], 0], ['2022-08-12 12:05:53.980', [5, 14, 21, 29, 37, 44, 52, 61, 65], 0], ['2022-08-12 12:06:54.167', [5, 16, 22, 29, 37, 45, 52, 61, 66], 0], ['2022-08-12 12:08:02.135', [6, 16, 23, 30, 38, 45, 53, 63, 66], 0], ['2022-08-12 12:09:03.322', [8, 16, 24, 31, 39, 45, 53, 63, 67], 0], ['2022-08-12 12:10:12.228', [8, 17, 24, 32, 40, 46, 54, 65, 68], 0], ['2022-08-12 12:11:12.524', [9, 18, 25, 32, 41, 47, 55, 65, 68], 0], ['2022-08-12 12:12:15.930', [9, 19

114 번째 시도:  [['2022-08-12 11:57:33.391', [9, 17, 26, 32, 40, 50, 53, 61], 0], ['2022-08-12 11:58:34.578', [10, 18, 26, 33, 40, 50, 54, 62], 0], ['2022-08-12 11:59:34.749', [10, 18, 26, 33, 41, 51, 55, 63], 0], ['2022-08-12 12:00:35.108', [11, 18, 27, 34, 41, 51, 55, 63], 0], ['2022-08-12 12:01:40.686', [1, 12, 19, 27, 35, 42, 51, 56, 63], 1], ['2022-08-12 12:02:44.357', [3, 12, 19, 27, 36, 42, 51, 57, 63], 0], ['2022-08-12 12:03:44.528', [3, 14, 20, 28, 37, 43, 52, 58, 65], 0], ['2022-08-12 12:04:51.825', [4, 14, 21, 28, 37, 44, 52, 60, 65], 0], ['2022-08-12 12:05:53.980', [5, 14, 21, 29, 37, 44, 52, 61, 65], 0], ['2022-08-12 12:06:54.167', [5, 16, 22, 29, 37, 45, 52, 61, 66], 0], ['2022-08-12 12:08:02.135', [6, 16, 23, 30, 38, 45, 53, 63, 66], 0], ['2022-08-12 12:09:03.322', [8, 16, 24, 31, 39, 45, 53, 63, 67], 0], ['2022-08-12 12:10:12.228', [8, 17, 24, 32, 40, 46, 54, 65, 68], 0], ['2022-08-12 12:11:12.524', [9, 18, 25, 32, 41, 47, 55, 65, 68], 0], ['2022-08-12 12:12:15.930', [9, 19

ConnectionError: HTTPConnectionPool(host='apis.data.go.kr', port=80): Max retries exceeded with url: /6410000/busarrivalservice/getBusArrivalItem?serviceKey=bwUUK6MEA6GnqDpo2ZS9ChN4VZ1nkUPXrryU80XxUcLqxB62pIGIPWXNdyzqgT20BHrw8zxWWk9rqiSQwTrbnQ%3D%3D&routeId=200000112&stationId=228000703 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001F1FF738940>: Failed to establish a new connection: [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다'))